In [1]:
import os
from pathlib import Path
import shutil

import cv2
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
data_dir = Path.cwd().parent / "syndata-generation" / "hawking_output_dir"
image_dir = data_dir / "images"
label_dir = data_dir / "annotations"
output_dir = data_dir / "labeled_test_data"
output_dir.mkdir(parents=True, exist_ok=True)

In [5]:
for image_path in image_dir.glob("*.jpg"):
    image = cv2.imread(str(image_path))
    h, w = image.shape[:-1]
    with open(label_dir / f"{image_path.stem.split('_')[0]}.txt", "r") as infile:
        boxes = [line.strip().split(" ") for line in infile.readlines()]
        for i, box in enumerate(boxes):
            boxes[i][1:] = map(float, box[1:])
    for box in boxes:
        x, y, width, height = map(int, box[1:])
        x = int(box[1] * w)
        y = int(box[2] * h)
        width = int(box[3] * w)
        height = int(box[4] * h)
        image = cv2.rectangle(image, (x - width // 2, y - height // 2), (x + width // 2, y + height // 2), (255, 0, 0), 3)
        # image = cv2.rectangle(image, (int(box[1]), int(box[2])), (int(box[3]), int(box[4])), (255, 0, 0), 3)
    cv2.imwrite(str(output_dir / image_path.name), image)

In [4]:
original_stems = [path.stem for path in image_dir.glob("*.jpg")]
filtered_stems = [path.stem for path in output_dir.glob("*.jpg")]

In [18]:
print(len(original_stems), len(filtered_stems))

221 131


In [20]:
removal_stems = list(set(original_stems) - set(filtered_stems))

In [23]:
for stem in removal_stems:
    os.remove(image_dir / f"{stem}.jpg")
    os.remove(label_dir / f"{stem}.txt")